# Leren: Programming assignment 5

**Student 1:**  <span style="color:red">Tycho Koster</span> (<span style="color:red">10667687</span>)<br>
**Student 2:** <span style="color:red">David Stap</span> (<span style="color:red">10608516</span>)<br>

In [178]:
import math
import numpy as np
from numpy import genfromtxt
import operator

# Question 1) a

In [179]:
# import X and y from csv file
def read_logistic_data(filename):
    my_data = genfromtxt(filename, delimiter=';')
    X = []
    y= []
    for i in range(len(my_data)):
        new_data = my_data[i]
        y.append([new_data[-1]])
        X.append(np.delete(new_data, -1))
    return X, y

X_train, y_train = read_logistic_data('digits123-1.csv')
X_test, y_test = read_logistic_data('digits123-2.csv')

In [180]:
'''
INPUT: 
- training data (features and class concatenated)

OUTPUT:
- dictionary where classes are keys, with training examples
  corresponding to that class as values.
'''
def separate_classes(data):
    classdict = dict()
    for i in range(len(data)):
        x = data[i]
        if (x[-1] not in classdict):
            classdict[x[-1]] = []
        classdict[x[-1]].append(x)
    
    return classdict

classes = separate_classes(np.concatenate((X_train, y_train), axis=1))

In [181]:
'''
INPUT:
- numpy array

OUTPUT:
- mean of values in array
- variance of values in array
'''
def mean_var(x):
    mean = sum(x)/len(x) 
    variance = sum(((x-mean)**2))/len(x)   
    return mean, variance    

In [182]:
'''
INPUT:
- class

OUTPUT:
- dictionary where classes are keys, with (mean, variance) list
  of training examples corresponding to correct class.
'''
def mean_var_class(c):
    mean_var_list = list()
    
    # loop over every attribute
    for i in range(len(classes[c][0])-1):
        mean_var_attribute = np.array([])
        # loop over every training example
        for j in range(len(classes[c])):
            mean_var_attribute = np.concatenate((mean_var_attribute, np.array([classes[c][j][i]])), axis=0)
        mean_var_list.append(mean_var(mean_var_attribute))
    
    return mean_var_list


# create dict: key = class, entry = list with mean and variance for every feature    
mean_var_dict = {1.0 : mean_var_class(1.0), 2.0: mean_var_class(2.0), 3.0 : mean_var_class(3.0)}

In [183]:
'''
INPUT:
- mean
- variance
- value for feature x

OUTPUT:
- probability density function
'''
def pdf(mean, variance, x):
    # ignore training examples with value 0 for mean and variance,
    # i.e. return p = 1, so that multiplying with p does not change value.
    if ((mean == 0) & (variance == 0)):
        p = 1
    else:
        p = (1/(math.sqrt(2*variance*math.pi)))*math.e**-(((x-mean)**2)/(2*variance))
    
    return p

In [184]:

'''
INPUT:
- list with classes
- test instance x

OUTPUT:
- dictionary where classes are keys, with p that x
  is in class as value.
'''
def p_x_given_class(classes, x):
    probabilities = {}
    for c in classes:     
        p = 1
        for i in range(len(x)):
            feature_train = x[i]
            feature_mean = mean_var_dict[c][i][0]
            feature_var = mean_var_dict[c][i][1]
            p *= pdf(feature_mean, feature_var, feature_train)
        probabilities[c] = p
            
    return probabilities

'''
INPUT:
- list with classes
- test instance x

OUTPUT:
- prediction for class
'''
def prediction(classes, x):    
    p_x_given_c = p_x_given_class(classes,x)
    
    # find n of training examples
    length = 0
    for i in classes:
        length += len(classes[i])
        
    # find value for P(x|c1) + P(x|c2) + ... + P(x|cn)
    divide_by = 0    
    for i in classes:
        divide_by += p_x_given_c[i] * len(classes[i])/float(length)
         
    for i in p_x_given_c:
        p_x_given_c[i] = ((p_x_given_c[i] * len(classes[i])/float(length)) / divide_by)
        
    return max(p_x_given_c.iteritems(), key=operator.itemgetter(1))[0]

print class_probabilities(classes, X_train[0])
print prediction(classes, X_train[0])

{1.0: 3.952058804347397e-47, 2.0: 2.1858205950974444e-64, 3.0: 1.8196239624211907e-63}
1.0


In [185]:
'''
INPUT:
- test data: features
- test data: correct classes

OUTPUT:
- n of correctly classified instances
- n of incorrectly classified instances
- list with index of incorrectly classified instances
'''
def evaluation_gaussian_nb(X_test, y_test):
    true = 0
    false = 0   
    false_instances = list()
    for i in range(len(X_test)):
        predicted = prediction(classes, X_test[i])
        actual = y_train[i][0]
        if predicted == actual:
            true += 1
        if predicted != actual:
            false += 1
            false_instances.append(i)
             
    return true, false, false_instances

true, false, false_instances = evaluation_gaussian_nb(X_test, y_test)

print 'Correct:   ', true
print 'False:     ', false
print 'Accurracy: ', round((float(true)/float(false + true)), 5)    

[array([  0.,   0.,   0.,  12.,  13.,   5.,   0.,   0.,   0.,   0.,   0.,
        11.,  16.,   9.,   0.,   0.,   0.,   0.,   3.,  15.,  16.,   6.,
         0.,   0.,   0.,   7.,  15.,  16.,  16.,   2.,   0.,   0.,   0.,
         0.,   1.,  16.,  16.,   3.,   0.,   0.,   0.,   0.,   1.,  16.,
        16.,   6.,   0.,   0.,   0.,   0.,   1.,  16.,  16.,   6.,   0.,
         0.,   0.,   0.,   0.,  11.,  16.,  10.,   0.,   0.]), array([  0.,   0.,   0.,   0.,  14.,  13.,   1.,   0.,   0.,   0.,   0.,
         5.,  16.,  16.,   2.,   0.,   0.,   0.,   0.,  14.,  16.,  12.,
         0.,   0.,   0.,   1.,  10.,  16.,  16.,  12.,   0.,   0.,   0.,
         3.,  12.,  14.,  16.,   9.,   0.,   0.,   0.,   0.,   0.,   5.,
        16.,  15.,   0.,   0.,   0.,   0.,   0.,   4.,  16.,  14.,   0.,
         0.,   0.,   0.,   0.,   1.,  13.,  16.,   1.,   0.]), array([  0.,   0.,   0.,   2.,  16.,  16.,   2.,   0.,   0.,   0.,   0.,
         4.,  16.,  16.,   2.,   0.,   0.,   1.,   4.,  12.,  16.,  12

# Question 1) b

In [186]:
# pretty print misclassified instances to obtain a better understanding of underlying issues.
c = classes
for i in false_instances:
    print class_probabilities(c, X_test[i])
    print 'prediction: ', prediction(c, X_test[i])
    print 'actual:     ',y_train[i][0]
    print '------------------------------------------------------------------------'    

{1.0: 8.7727579919187929e-68, 2.0: 7.6757727494372748e-66, 3.0: 0.0}
prediction:  2.0
actual:      1.0
------------------------------------------------------------------------
{1.0: 3.0013990405093858e-79, 2.0: 2.3129165503814655e-70, 3.0: 0.0}
prediction:  2.0
actual:      1.0
------------------------------------------------------------------------
{1.0: 3.2307747381093141e-70, 2.0: 1.6815889701900517e-59, 3.0: 4.3221550669138758e-96}
prediction:  2.0
actual:      1.0
------------------------------------------------------------------------
{1.0: 1.7349709895829546e-71, 2.0: 4.0829056283834716e-48, 3.0: 3.1084867736730183e-71}
prediction:  2.0
actual:      1.0
------------------------------------------------------------------------
{1.0: 1.0158273915718127e-97, 2.0: 3.1710937831898381e-111, 3.0: 1.5007752556678841e-86}
prediction:  3.0
actual:      1.0
------------------------------------------------------------------------
{1.0: 7.4253158850782486e-159, 2.0: 1.79297024732864e-73, 3.0:

### Conclusion 

False per class:
1     17/68
2     45/68
3     6/68

Conclusion: class 2 gets misclassified most often by far. Also, the algorithm seems to assign a relatively high probability (to the incorrect class) when misclassifying class 2.

Now we have to find out where the algorithm gets it wrong.

class 2 misclassified as:
1     5/45
2     40/45

Conclusion: the main problem is that class 2 gets misclassified as class 3. This specific type of error accounts for 40 out of the 68 errors. Besides being the most frequent error, these examples also have the highest probability of the misclassified instances.  

In almost all the cases where class 2 gets misclassified as class 3, the probability for class 1 is zero (or many orders of magnitude smaller than the other probabilities), and the probability for class 2 and class 3 are relatively close to each other ('relatively' meaning within 10 orders of magnitude).

Class 2 consists of 96 training examples. Both class 1 and class 2 consist of 102 training examples. When training a Gaussian Naive Bayes classifier on this slightly uneven spread dataset, there is a bias for class 1 / class 2 (because there are more instances of these classes to be found than class 2). To improve accurracy, we can add more training examples of class 2 so that this bias will disappear.

Another conclusion that can be drawn: the algorithm finds it more difficult to distinguish between 2 and 3 than between 2 and 1. The shape for the number 1 is very easy: a straight line. The shape of numbers 2 and 3 have more overlap, therefore it is harder to distinguish between these classes. Solution: add more training examples to better learn the different characteristics. 

# Question 2

In [212]:
from sklearn import linear_model
import random


# Creates result array y with dat x
def make_xy(data):
    y_data = []
    x_data = []
    for x in range(0, len(data)):
        y_data.append([data[x][-1]])
        x_data.append(data[x][:-1])
    return x_data, y_data

def split_data(X_data):
    percent60 = int(len(X_data)*0.6)
    percent20 = int(len(X_data)*0.2)
    percent80 = percent60 + percent20
    train_data = X_data[:percent60]
    X_train, y_train = make_xy(train_data)
    cross_data = X_data[percent60:percent80]
    X_cross, y_cross = make_xy(cross_data)
    test_data = X_data[percent80:]
    X_test, y_test = make_xy(test_data)
    return X_train, y_train, X_cross, y_cross, X_test, y_test


def cross_validation(shuffled_data):
    #Logistic regression scikit
    reg = linear_model.LinearRegression()
    # Shuffle array so the split is random.
    # Split the data in a training set, cross validation set and a test set.
    X_train, y_train, X_cross, y_cross, X_test, y_test = split_data(shuffled_data)
    # Train the classifiers on the training set
    # Our Guassian classifier.
    classes = separate_classes(np.concatenate((X_train, y_train), axis=1))
    mean_var_dict = {1.0 : mean_var_class(1.0), 2.0: mean_var_class(2.0), 3.0 : mean_var_class(3.0)} 
    # Scikits logistic regression classifier
    reg.fit(X_train, y_train) 
    correct_cross, incorrect_cross, false_instances_cross = evaluation_gaussian_nb(X_cross, y_cross)
    correct_test, incorrect_test, false_instances_test = evaluation_gaussian_nb(X_test, y_test)
    print 'Accurracy our Gaussian cross set:   ', round((float(correct_cross)/float(incorrect_cross + correct_cross)), 5)  
    print 'Accurracy our Gaussian test set:   ', round((float(correct_test)/float(incorrect_test + correct_test)), 5)  
    print 'Accuracy logistic regression scikit cross set: ', reg.score(X_cross, y_cross)
    print 'Accuracy logistic regression scikit test set: ', reg.score(X_test, y_test)
    

In [210]:
X_data, y_data = read_logistic_data('digits123-1.csv')
data_array = np.concatenate((X_data, y_data), axis=1)
shuffled_data = random.sample(data_array, len(data_array))

In [213]:
print 'Test set digits123-1 split in 3 for cross validation:'
cross_validation(shuffled_data)

Test set digits123-1 split in 3 for cross validation:
Accurracy our Gaussian cross set:    0.28333
Accurracy our Gaussian test set:    0.43333
Accuracy logistic regression scikit cross set:  0.85670846221
Accuracy logistic regression scikit test set:  0.886900200954


### Discussion

We noticed that our Gaussian Naive Bayes classifier has a very low accuracy, compared to the scikit logistic regression classifier. There is no big difference between the cross set and the test set. The test set is predicted a bit better with both classifiers.

We think that the problem was the training set. It was very small compared to what we used in question 1, where the accuracy was way higher when trained on the full dataset digits123-1 and tested on digits123-2. 

Because digits123-1 is sorted by class, we had to randomize our dataset to get a training set with multiple classes. There is a problem with this randomization, because it still makes it possible to have a training set with only 2 classes instead of 3. The chances of this happening are small. This makes it hard for a classifier to classify the new third class. 

Our classifier needs more training data to come close to the accuracy of the logistic regression classifier used by scikit. So the logistic regression classifier is better with this dataset.